In [465]:
import torch
import torchvision
import torch.nn as nn

In [466]:
def anchor_type(size = (4,8,16),ratio = (2,1,0.5)):
    anchor_types = []
    for i in size:
        for j in ratio:
            anchor_types.append((int(i),int(i*j)))
    return anchor_types

In [467]:
def anchorGen(feat_maps):
    center_points = []
    for feat_map in feat_maps:
        cp=[]
        for i in range(feat_map.shape[2]):
            for j in range(feat_map.shape[2]):
                cp.append((i,j))
        center_points.append(cp)
    return center_points

In [468]:
def usable_anchors(feat_maps,size=(4,8,16),ratio=(2,1,0.5)):
    anchor_types = anchor_type()
    useful_anchors = []
    x = y = feat_maps.shape[2]
    for i in range(x):
        for j in range(y):
            flag = True
            for k in anchor_types:
                if((i - (k[0]/2)) < 0 or (j - (k[1]/2)) < 0 or (i + (k[0]/2)) > x or (j + (k[1]/2)) > y):
                    flag = False
                else:
                    useful_anchors.append((i,j,k[0]/2,k[1]/2))
    return useful_anchors

In [469]:
def convert(anchors_to_be_converted):
    converted_anchors = []
    for i in anchors_to_be_converted:
        xmax = (i[0] + i[2]/2)
        ymax = (i[1] + i[3]/2)
        xmin = (i[0] - i[2]/2)
        ymin = (i[1] - i[3]/2)
        converted_anchors.append((xmax,ymax,xmin,ymin))
    return converted_anchors

In [470]:

def iou_threshold(feat_maps, ground_truth, threshold = 0.49):
    anchors = usable_anchors(feat_maps)
    anchors = convert(anchors)
    ground_truth = convert(ground_truth)
    f_anchors=[]
    finalized_anchors = []
    scores=[]
    for i in anchors:
        for j in ground_truth:
            area_intersection = (min(i[0],j[0])-max(i[2],j[2])) * (min(i[1],j[1])-max(i[3],j[3]))
            area_box1 = (i[0]-i[2])*(i[1]-i[3])
            area_box2 = (j[0]-j[2])*(j[1]-j[3])
            iou = area_intersection/(area_box1+area_box2-area_intersection+1e-6)
            if iou>threshold and iou<1 and area_intersection>0 and (min(i[0],j[0])-max(i[2],j[2]))>0:
                f_anchors.append(i)
                scores.append(iou)
    return f_anchors,scores